In [1]:
#imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import pandas as pd

In [2]:
#set up api key variables
keysfile = open('apikeys','r')
cl_id = keysfile.readline().strip()
cl_sec = keysfile.readline().strip()

#set up spotify object
auth_manager = SpotifyClientCredentials(client_id=cl_id, client_secret=cl_sec)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [19]:
#requests to get all of moby's albums
moby_uri = 'spotify:artist:3OsRAKCvk37zwYcnzRf5XF'

results = sp.artist_albums(moby_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])


48

In [ ]:
# only released in the US (which removes a bunch of duplicates)
albums = [album for album in albums if 'US' in album['available_markets'] ]

# remove duplicates
temp=[]
for album in albums:
    if album['name'].lower() not in [a['name'].lower() for a in temp]:
        temp.append(album)
albums = temp

# remove specific albums
omit = ["Reprise (Commentary Version)","Porcelain (Gekürzte Lesung)","Destroyed","Wait for Me","Wait for Me (Deluxe Version)"]
albums = [album for album in albums if album['name'] not in omit]

# for album in albums:
#     print(album['name'])
# len(albums)


In [40]:
#get all the track ids and names from all of the albums
track_ids = []
track_names = []
track_albums = []

for album in albums:
    tracks = sp.album_tracks(album['id'])['items']
    track_ids.extend([track['id'] for track in tracks])
    track_names.extend([track['name'] for track in tracks])
    track_albums.extend([album['name']] * len(tracks))


581

In [52]:
data = []
indexes = list(range(0,len(track_ids),100)) + [len(track_ids)]
for i in range(len(indexes)-1):
    data.extend(sp.audio_features(track_ids[indexes[i]:indexes[i+1]]))


In [53]:
len(data)

581

In [58]:
#put it in a dataframe with names, drop unnecessary fields, export
df = pd.DataFrame(data)
df.insert(0,'name',track_names,True)
df.insert(1,'album',track_albums,True)
df.drop(['type','uri','track_href','analysis_url'], axis=1, inplace=True)
df.to_csv('msd.csv')

,name,album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,Heroes - planningtorock Remix,Reprise - Remixes,0.483,0.978,4,-4.760,0,0.0484,0.000232,0.606,0.1120,0.7520,124.009,3d5AEtmwhwm9Sq0VWSK5qJ,267229,4
1,Porcelain - Christian Löffler Remix,Reprise - Remixes,0.616,0.340,4,-13.648,0,0.0298,0.340000,0.727,0.1270,0.3300,92.010,1o9e4RnIIbc9qhS0BszyjJ,232358,4
2,Go - Moby's Trophy Remix,Reprise - Remixes,0.608,0.890,4,-9.753,0,0.0538,0.077600,0.882,0.0908,0.2330,124.996,7uHV6bXy6L7cwEDNsfdyUh,355293,4
3,Porcelain - Bambounou Remix,Reprise - Remixes,0.483,0.850,2,-7.501,1,0.0490,0.034000,0.820,0.1380,0.5160,160.026,4VQjSJF6TGtJxHo7Tg8Li8,317569,4
4,Go - Anfisa Letyago Remix,Reprise - Remixes,0.674,0.958,11,-6.167,0,0.0404,0.000540,0.897,0.1070,0.5360,126.989,2DILDB3AW4KcxN1zyOVw3s,338741,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,Next Is the E - Victory Mix,Next Is the E,0.502,0.911,2,-8.517,1,0.0500,0.000029,0.435,0.1460,0.2310,147.024,3L29xjypuqXGc0u6XSb9wP,345000,4
577,Next Is the E - Synthe Mix,Next Is the E,0.680,0.905,2,-8.084,1,0.0466,0.001070,0.937,0.0824,0.3920,122.925,3VBu5kVxEEXXgjh133XNuD,420000,4
578,Next Is the E - I Feel It,Next Is the E,0.596,0.977,7,-7.811,1,0.0387,0.001090,0.368,0.0908,0.6290,136.964,34sexw77fyoNCuUiHbte32,359000,4
579,Thousand,Next Is the E,0.206,0.892,11,-8.326,0,0.0565,0.000321,0.973,0.1120,0.0494,126.791,5p2LNkxerTz3iMHVzhnC9z,266000,3
